In [62]:
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt 
from mlxtend.frequent_patterns import apriori,association_rules

In [49]:
data = pd.read_csv(r"F:\anlas\Python ile Makine Öğrenmesi\ecommerceData.csv",encoding="cp1252")
data2 = data.copy()
data2.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [50]:
# Missing values & incorrect data
print(data2.isnull().sum())
# We can't adjust CustomerID, I will delete
data2.dropna(inplace=True)

data2 = data2[~data2["InvoiceNo"].str.startswith('C')]
data2.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64


InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [51]:
# EDA
data2["Country"].value_counts()
# Acc. to apriori I will use one country. 
# The UK is the biggest e-com I will use it to do that

Country
United Kingdom          354345
Germany                   9042
France                    8342
EIRE                      7238
Spain                     2485
Netherlands               2363
Belgium                   2031
Switzerland               1842
Portugal                  1462
Australia                 1185
Norway                    1072
Italy                      758
Channel Islands            748
Finland                    685
Cyprus                     614
Sweden                     451
Austria                    398
Denmark                    380
Poland                     330
Japan                      321
Israel                     248
Unspecified                244
Singapore                  222
Iceland                    182
USA                        179
Canada                     151
Greece                     145
Malta                      112
United Arab Emirates        68
European Community          60
RSA                         58
Lebanon                     45


In [52]:
focusedData = data2[data2["Country"] == "United Kingdom"]
focusedData = focusedData.iloc[:,[0,2,3]] # some feature is not about apriori
focusedData

,InvoiceNo,Description,Quantity
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6
1,536365,WHITE METAL LANTERN,6
2,536365,CREAM CUPID HEARTS COAT HANGER,8
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6
...,...,...,...
541889,581585,FAIRY TALE COTTAGE NIGHT LIGHT,12
541890,581586,LARGE CAKE STAND HANGING STRAWBERY,8
541891,581586,SET OF 3 HANGING OWLS OLLIE BEAK,24
541892,581586,RED RETROSPOT ROUND CAKE TINS,24


In [ ]:
# Grouping 
basket = focusedData.groupby(["InvoiceNo","Description"])["Quantity"].sum().unstack().reset_index().fillna(0).set_index(keys="InvoiceNo")

basket = basket.applymap(lambda x: 0 if x <=0 else 1) # this is for apriori.
basket

C:\Users\kemal\AppData\Local\Temp\ipykernel_11316\4188750419.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket = basket.applymap(lambda x: 0 if x <=0 else 1)


Description,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,TOADSTOOL BEDSIDE LIGHT,...,ZINC STAR T-LIGHT HOLDER,ZINC SWEETHEART SOAP DISH,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE KITCHEN ORGANISER,ZINC WIRE SWEETHEART LETTER TRAY
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536366,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536367,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536368,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536369,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581582,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
581583,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
581584,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [64]:
df1 = apriori(basket,min_support=0.02,use_colnames=True)
df2 = association_rules(df1,metric="confidence",num_itemsets=len(data["Description"].unique()))
print(df2)

                                         antecedents  \
0                   (PINK REGENCY TEACUP AND SAUCER)   
1  (PINK REGENCY TEACUP AND SAUCER, GREEN REGENCY...   
2  (PINK REGENCY TEACUP AND SAUCER, ROSES REGENCY...   

                          consequents  antecedent support  consequent support  \
0   (GREEN REGENCY TEACUP AND SAUCER)            0.029611            0.036759   
1  (ROSES REGENCY TEACUP AND SAUCER )            0.024266            0.040723   
2   (GREEN REGENCY TEACUP AND SAUCER)            0.023004            0.036759   

    support  confidence       lift  representativity  leverage  conviction  \
0  0.024266    0.819473  22.293137               1.0  0.023177    5.335706   
1  0.020482    0.844059  20.726763               1.0  0.019494    6.151553   
2  0.020482    0.890339  24.221015               1.0  0.019636    8.783841   

   zhangs_metric   jaccard  certainty  kulczynski  
0       0.984289  0.576320   0.812583    0.739802  
1       0.975423  0.460189   0.83

In [68]:
df2.iloc[:,[0,1,2,3]]

,antecedents,consequents,antecedent support,consequent support
0,(PINK REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.029611,0.036759
1,"(PINK REGENCY TEACUP AND SAUCER, GREEN REGENCY...",(ROSES REGENCY TEACUP AND SAUCER ),0.024266,0.040723
2,"(PINK REGENCY TEACUP AND SAUCER, ROSES REGENCY...",(GREEN REGENCY TEACUP AND SAUCER),0.023004,0.036759
